In [1]:
# Install and update the required library
%%capture
!pip install unsloth

# Load Model

In [2]:
# Import necessary libraries from Unsloth and PyTorch
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

# Load the Llama-3.2 model with 4-bit quantization for efficient memory use
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Mllama vision patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/375k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

preprocessor_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

In [3]:
# Configure LoRA (Low-Rank Adaptation) fine-tuning settings
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # False if not finetuning vision layers
    finetune_language_layers   = True,  # False if not finetuning language layers
    finetune_attention_modules = True,  # False if not finetuning attention layers
    finetune_mlp_modules       = True,  # False if not finetuning MLP layers

    r = 16,            # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,   # Recommended alpha == r at least
    lora_dropout = 0,  # No dropout for LoRA
    bias = "none",
    random_state = 3407,
    use_rslora = False,   # Do not use rank-stabilized LoRA
    loftq_config = None,  # No use of LoftQ
)

Unsloth: Making `model.base_model.model.language_model` require gradients


# Load Dataset

In [7]:
# Load the dataset (text-only) from Hugging Face
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

In [8]:
# Display dataset structure
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [9]:
# Display the first example in the dataset
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.'},
  {'from': 'gpt

In [10]:
# Example format of dataset conversations (Human-GPT interactions)
"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
"""

"\n<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI'm great thanks!<|eot_id|>\n"

In [11]:
# Function to convert dataset into the required conversation format
def convert_to_conversation(sample):
    new_conversation = []
    for init_conversation in sample['conversations']:
      if init_conversation['from'] == 'human':
         role = 'user'
      elif init_conversation['from'] == 'gpt':
         role = 'assistant'
      else:
        continue
      new_conversation.append(
           { "role": role,
             "content" : [
                {"type" : "text",  "text"  : init_conversation['value']},
             ]
           },
      )
    return {"messages" : new_conversation}

In [12]:
# Apply conversion to the dataset
converted_dataset = [convert_to_conversation(sample) for sample in dataset]

In [13]:
# Display the first converted conversation
converted_dataset[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming la

In [14]:
# Extracting the first instruction from the dataset
instruction = dataset[0]["conversations"][0]['value']
instruction

'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.'

# Plain text Inference test (no image input)

In [15]:
# Prepare model for inference mode (after fine-tuning)
FastVisionModel.for_inference(model) # Enable for inference!

image = None  # No image input since the dataset is text-only

# Format the user prompt as required for the model
messages = [
    {"role": "user", "content": [
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)

# Display formatted input text
input_text

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Feb 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code th

In [16]:
# Tokenize the input text for model inference
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

# Generate model output using the provided text input
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

**Boolean Operators**

Boolean operators are used to evaluate conditions and produce a result. The most common boolean operators are:

- `NOT` (!, NOT): Used to invert a boolean value. In many programming languages, `NOT` is also used for negating bitwise values, but it works in a similar way in boolean operations.

- `AND` (&&, AND): Returns `true` if both operands are `true`. In many programming languages, this operator is also used for bitwise operations.

- `OR` (||, OR): Returns `true` if at least one operand is `true`.

- `EQV` (XOR, ^, #, ==,!=, ~=, not identical in Ruby): In bitwise operations it returns `1` if either of the bits is `1` (ones), otherwise `0` (zero) in an exclusive disjunctive set. In boolean operations it returns `1` if exactly one of the bits is `1` (one).

Examples in Python:
```python
x = 5
y = 10

# Using AND
print(x > 2 and x < 10)  # Returns: True
print(y > 20 or x < 15)  # Returns: True

# Using XOR in Python
print(x > 10!= x > 5)  # Returns: True

# Usi

In [18]:
# Inference for image+text data input
import requests
from PIL import Image

# Tokenize the input text for model inference
image = Image.open(requests.get("https://llava-vl.github.io/static/images/view.jpg", stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
          {"type": "text", "text": "What are the things I should be cautious about when I visit this place? What should I bring with me?"},
          {"type" : "image", "image" : image}
        ]
    }
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)

inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

# Generate model output using the provided text input
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

**The Ultimate Destination for Nature Lovers and Adventure Seekers**

**Safety Considerations:**

*   Be cautious of the steep terrain and uneven pathways, particularly during rainy weather.
*   Watch out for wildlife, including bears, cougars, and other animals that may be present in the area.
*   Respect the park's rules and regulations, including staying on designated trails and not feeding the wildlife.

**Packing Essentials:**

*   Comfortable hiking shoes or boots
*   Rain gear (jacket, pants, umbrella)
*   Warm clothing (fleece, hat, gloves)
*   Water bottle
*   Snacks
*   Map and compass (or GPS device)
*   First aid kit
*   Sunscreen and insect repellent
*   Camera
*   Binoculars

**Additional Tips:**

*   Check the weather forecast before heading to the park, and be prepared for changing conditions.
*   Bring a small backpack or daypack to carry your essentials.
*   Don't forget to bring your sense of adventure and a camera to capture the stunning views!
*   The park's facili

# Fine tune the model

In [19]:
# Convert dataset to Hugging Face Dataset format
from datasets import Dataset
my_dataset = Dataset.from_dict({"dataset": converted_dataset})

print(my_dataset)

Dataset({
    features: ['dataset'],
    num_rows: 100000
})


In [20]:
# Function to format prompts for training
def formatting_prompts_func(examples):
    try:
        convos = examples["dataset"]
        texts = [tokenizer.apply_chat_template(convo['messages'], tokenize = False, add_generation_prompt = False) for convo in convos]
        return { "text" : texts, }
    except:
        print(examples)
        raise

# Apply formatting to dataset
dataset = my_dataset.map(formatting_prompts_func, batched = True,)

# Display formatted dataset example
dataset[2]['text']

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Feb 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions.\n\nFurthermore, discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. Finally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nBoolean operators are logical operators used to combine or manipulate boolean values in programming. They allow you to perform comparisons and create complex logical expressions. The three main boolean oper

In [21]:
# Preprocessing functions for dataset tokenization
from functools import partial

def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizes dataset batch
    """

    return tokenizer(
        images=None,
        text=batch["text"],
        max_length = max_length,
        truncation = True,
    )

def preprocess_dataset(tokenizer, max_length, seed, my_dataset):
    """
    Tokenizes dataset for fine-tuning
    """
    columns_names = my_dataset.column_names
    columns_names.append('text')

    # Apply preprocessing to each batch of the dataset
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    my_dataset = my_dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = columns_names,
    )

    # Filter out samples that have "input_ids" exceeding "max_length"
    my_dataset = my_dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    my_dataset = my_dataset.shuffle(seed = seed)

    return my_dataset

In [22]:
# Set preprocessing parameters
max_length = 2048
seed = 33

# Preprocess dataset
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)

print(preprocessed_dataset)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 97972
})


In [23]:
# Custom data collator for text fine-tuning
import torch

class TextDataCollator:
    def __init__(self, model, tokenizer, max_length=2048):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, examples):
        # Pad or truncate input_ids and attention_mask
        input_ids = [ex["input_ids"][:self.max_length] for ex in examples]
        attention_mask = [ex["attention_mask"][:self.max_length] for ex in examples]

        # Pad sequences to max_length
        input_ids = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(ids) for ids in input_ids],
            batch_first=True,
            padding_value=0
        )

        attention_mask = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(mask) for mask in attention_mask],
            batch_first=True,
            padding_value=0
        )

        # Add labels (same as input_ids for language modeling)
        labels = input_ids.clone()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

In [24]:
# Fine-tuning configuration and training using SFTTrainer
from unsloth import is_bf16_supported
from trl import SFTTrainer, SFTConfig


FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator =  TextDataCollator(model, tokenizer),
    train_dataset = preprocessed_dataset,

    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        # num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        max_seq_length = 2048,
    ),
)

In [25]:
# Start training
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 97,972 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 52,428,800
🦥 Unsloth needs about 1-3 minutes to load everything - please wait!


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.776500
2,1.533800
3,1.531000
4,0.929400
5,1.018700
6,1.047200
7,0.904700
8,0.947800
9,0.840500
10,0.768900


In [26]:
# Fine-tuned Model Inference Test
FastVisionModel.for_inference(model)

image = None
instruction= "Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages."

messages = [
    {"role": "user", "content": [
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True
    )


inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Here is an example implementation in Python of the boolean operators and their usage:


```python
def bool_operation(x, y, op):
    result = False
    if x > 5 and y > 3:
        result = True
        if y >= 4:
            result = True
        elif y <= 2:
            result = True
        elif y == 0:
            result = False
            break

    return result
```



Now, we will look at the concept of truthiness and falsiness in programming languages:


```python
def truthiness_and_falsiness(x, y):
    truthiness_value = True
    falsiness_value = False
    print("Truthiness:", x > 3 and y == 0 and (x + y!= 0))
    return (x > 0, y == 0, x + y == 0)
```



We will also look at the concept of boolean operators:


```python
def boolean_operator(x, y):
    result = False
    if x == 0:
        result = False
    elif x > 0 and y > 0:
        result = True
    else:
        result = False
    return result
```



We will also look at the concept of short-circuit evaluation:


```py

In [27]:
# Inference for image+text data input
# Tokenize the input text for model inference
image = Image.open(requests.get("https://llava-vl.github.io/static/images/view.jpg", stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
          {"type": "text", "text": "What are the things I should be cautious about when I visit this place? What should I bring with me?"},
          {"type" : "image", "image" : image}
        ]
    }
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)

inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

# Generate model output using the provided text input
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

It looks like this place could be an outdoor location near a lake in the mountains. This is a beautiful area with plenty of scenic spots but it's essential to be mindful of safety, especially with the presence of water. Here are some tips to consider before and during your visit:

1. Keep your valuables secure and close by.
2. Swim in designated areas only and avoid areas marked off.
3. Be mindful of your surroundings and watch your belongings closely.
4. Ensure you know what you need beforehand and do not leave them behind.
5. Use reputable vendors when planning transportation.
6. Be respectful of the land and any property, and never disturb it if it belongs to another person.
7. Dress modestly and avoid showing your midriff or knees.
8. Respect other tourists in the area.

As for what to bring, consider bringing: 

1. Comfortable shoes or sneakers, as there will likely be rough paths to walk on.
2. Comfortable clothes with breathable fabrics like cotton or linen.
3. Sunscreen, hats, 